In [ ]:
# This code was written by Luka Skolc (ETHZ) under the supervision of Krzysztof Barczynski (PMOD/WRC & ETHZ),
# Nils Janitzek (PMOD/WRC & ETHZ) and Louise Harra (PMOD/WRC & ETHZ) in the scope of the ETH Studio 
# Davos Internship in 2023. The work on the code began in March 2023.

In [1]:
# This is the file with the functions for analysis of detected objects. The file is organized into 
# subsections which are separated by a ### line. Below the line are written the functions in that
# subsection.

#################################################################################################### 
#################################################################################################### 
# com_t(shape, intens_t)
# com_footpoint(objc, intensities)

def com_t(shape, intens_t): # computes the "centre of mass" for a given shape by weighting with intensity
    # "shape" is a set of coordinates {(yi, xi)} 
    # "intens_t" are the intensities at a fixed time frame
    
    total = 0 # sum of intensities over all pixels
    y_com = 0
    x_com = 0
    
    for coord in shape: # iterate across all pixels in the shape
        (yi, xi) = coord
        inte = intens_t[yi, xi] # we do normal weighting without any powers (see "inertia" function)
        y_com += yi * inte
        x_com += xi * inte
        total += inte
        
    return (y_com / total, x_com / total) # this is the instantaneous COM

    
def com_footpoint(objc, intensities): # calculate the position of the footpoint's COM
    if len(objc[1:]) < 10: # footpoint is the COM from the first frame
        return com_t(objc[1], intensities[:, :, objc[0]])
    
    else: # footpoint is the COM of the first two frames summed together (key = 'sum')
        (coordinates, intens_sum) = time_sum(objc[:3], intensities) # time sum of the first two frames
        total = np.sum(intens_sum) # sum of intensities over all pixels the object occupied in the first 
                                   # two frames
        y_com = 0 # the footpoint's COM
        x_com = 0
        
        for j in range(len(coordinates)): 
            (yi, xi) = coordinates[j]
            inte = intens_sum[j]
            y_com += yi * inte
            x_com += xi * inte
        
        return (y_com / total, x_com / total)


#################################################################################################### 
# object_com_traj(objc, intensities)
# object_edge_traj(objc, intensities)
# object_com_masked_traj(objc, intensities, traj_type)

def object_com_traj(objc, intensities): # returns an array of the object's COM's trajectory in time
    trajectory = np.zeros((len(objc[1:]), 2)) # here we store the COM trajectory

    for i in range(len(objc[1:])):
        (y_com, x_com) = com_t(objc[1 + i], intensities[:, :, objc[0] + i])
        trajectory[i, 0] = y_com
        trajectory[i, 1] = x_com
    
    return trajectory # type = float which is good for fitting. If we want to draw an imshow plot of
                      # the COM, we should first round and then convert to int type

def object_edge_traj(objc, intensities): # returns the trajectory of the pixel that is furthest away from
    # the COM of the first frame when the object appears.
    # we expect this trajectory to be a straight line for a jet and chaotic for flares. In reality,
    # the motion isn't nice even for some jets. The "expand" method is much more robust.
    
    (yc_0, xc_0) = com_footpoint(objc, intensities) # footpoint COM
    
    if len(objc[1:]) < 10: # short-lived objects
        trajectory = np.zeros((len(objc[1:]), 2)) # here we store the trajectory of the edge
        index0 = 2
    
    else:
        trajectory = np.zeros((len(objc[2:]), 2))
        index0 = 3
    
    trajectory[0, 0] = yc_0 # footpoint is the first point in the trajectory
    trajectory[0, 1] = xc_0
        
    for i in range(len(objc[index0:])):
        max_dist = 0

        for (y, x) in objc[index0 + i]: # over all coordinates in one time frame
            dist = np.sqrt((y - yc_0)**2 + (x - xc_0)**2) # usual Euclidean distance

            if dist > max_dist:
                max_dist = dist
                (y_edge, x_edge) = (y, x)

        trajectory[1 + i, 0] = y_edge
        trajectory[1 + i, 1] = x_edge

    return trajectory # type float
    
    
def object_com_masked_traj(objc, intensities, traj_type): # Calculates COM of the expanding part of 
    # the object. it is more robust than checking the most distant pixel (see function "edge_path")
    # "traj_type" can be 'com_masked' or 'expand'
    
    (yc_0, xc_0) = com_footpoint(objc, intensities) # footpoint COM
    
    if len(objc[1:]) < 10: # short-lived objects
        trajectory = np.zeros((len(objc[1:]), 2)) 
        mask = objc[1] # the initial mask are the pixels the object occupies when it first appears.
        index0 = 2     # For a jet, that should correspond to the footpoint
        
    else: # Long-lived objects
        trajectory = np.zeros((len(objc[2:]), 2))        
        mask = objc[2].union(objc[1])
        index0 = 3
        
    trajectory[0, 0] = yc_0
    trajectory[0, 1] = xc_0
    
    for i in range(len(objc[index0:])):
        remainder = objc[index0 + i].difference(mask) # Only consider pixels that weren't in any 
                                                      # previous frame.
        if remainder != set():
            (y_com, x_com) = com_t(remainder, intensities[:, :, objc[0] + (index0 - 1) + i])
            trajectory[1 + i, 0] = y_com
            trajectory[1 + i, 1] = x_com

        if remainder == set():
            return trajectory[:1 + i, :] # The length of the returned trajectory may be smaller than
                                         # the lifetime of the object
        if traj_type == 'expand':
            mask = mask.union(remainder) # Enlarge the mask by all points from this frame

    return trajectory # Type float


####################################################################################################    
# time_union(objc)
# time_sum(objc, intensities)
# time_max(objc, intensities)
# cubic_time(objc)
# cubic_intensity(objc, intensities)

def time_union(objc): # All the pixels on which the object ever was, joined into one set
    return set().union(*objc[1:])


def time_sum(objc, intensities): # Time sum of intensities for object "objc"
    coordinates = list(time_union(objc)) # All coordinates the object ever occupied
    # "coordinates" is a list of tuples (y, x)
    lenn = len(coordinates)      # Number of coordinates in the union
    intens_sum = np.zeros(lenn)  # The summed intensities at those coordinates. Type?
    t1 = objc[0]
    
    for i in range(len(objc[1:])): # Over all the times the objects existed
        for coord in objc[1 + i]:  # Over all the coordinates at a fixed time
            for j in range(lenn):  # Could be done more efficiently with sorting
                if coord == coordinates[j]:
                    intens_sum[j] += intensities[coord[0], coord[1], t1 + i]
    
    return (coordinates, intens_sum)


def time_max(objc, intensities): # Max intensity over time for each pixel the object ever occupied
    coordinates = list(time_union(objc)) # All coordinates the object ever occupied
    lenn = len(coordinates)              # Number of coordinates in the union
    intens_max = np.zeros(lenn) 
    t1 = objc[0]
    
    for i in range(len(objc[1:])): # Over all the times the objects existed
        for coord in objc[1 + i]:  # Over all the coordinates at a fixed time
            for j in range(lenn):
                if coord == coordinates[j]:
                    if intensities[coord[0], coord[1], t1 + i] > intens_max[j]:
                        intens_max[j] = intensities[coord[0], coord[1], t1 + i]
    
    return (coordinates, intens_max)


def cubic_time(objc): # Cubic time is adding the number of pixels the object occupies at each frame
                      # Over all the times it existed.
    cubt = 0          # It is a different concept than the time union!
    for i in range(len(objc[1:])):
        cubt += len(objc[1 + i])
        
    return cubt


def cubic_intensity(objc, intensities): # Sum of intensities over all active pixels at all times, for 
                                        # one object.
    (coords, intens_sum) = time_sum(objc, intensities)
    return np.sum(intens_sum)


####################################################################################################    
# inertia(objc, intensities, power, key)

def inertia(objc, intensities, power, key): # Calculates the moment of inertia evls of object "objc"
    if key == 'sum':  # Take sum of intensities over time at each pixel
        (coordinates, intenss) = time_sum(objc, intensities)
        
    if key == 'max':  # Take max intensity over time at each pixel
        (coordinates, intenss) = time_max(objc, intensities)
        
    if key == 'constant': # Constant density across the shape
        union = time_union(objc)
        (coordinates, intenss) = (list(union), np.ones(len(union)))
    
    N = len(coordinates)
    
    total = 0 # Sum of intensities over all pixels
    y_com = 0
    x_com = 0

    for i in range(N): # Iterate across all pixels in the shape
        (yi, xi) = coordinates[i]
        inte = intenss[i] **power   # The intensity is scaled by an exponent "power".
        y_com += yi * inte          # We found "power"=1 is optimal but left it free for generality
        x_com += xi * inte          # as ML may (unlikely) optimize it differently later
        total += inte
        
    (y_com, x_com) = (y_com / total, x_com / total) # This is the COM of the intens_sum
    
    I = np.zeros((2, 2))
    
    for i in range(N):
        I[0, 0] += (intenss[i]**power) * (coordinates[i][0] - y_com)**2
        I[1, 1] += (intenss[i]**power) * (coordinates[i][1] - x_com)**2
        I[1, 0] += -(intenss[i]**power) * (coordinates[i][0] - y_com) * (coordinates[i][1] - x_com)
        I[0, 1] += -(intenss[i]**power) * (coordinates[i][0] - y_com) * (coordinates[i][1] - x_com)
        
    a = I[0, 0]
    b = I[1, 1]
    c = I[1, 0] # Since it's a 2x2 matrix, we can diagonalize it by solving a quadratic equation
    
    evl1 = (a + b + np.sqrt((a + b)**2 - 4*(a*b - c**2))) / 2
    evl2 = (a + b - np.sqrt((a + b)**2 - 4*(a*b - c**2))) / 2
    
    return (min(evl1, evl2), max(evl1, evl2)) # Returns ordered eigenvalues


####################################################################################################
# rotate(y, x, phi)
# line_fit(traj)
# manual_regression(traj)
# default_angle_twostep(traj, com0, default)
# default_angle_multistep(traj, com0, default, tolerance)  

def rotate(y, x, phi): # Rotates the vector [x, y] by an angle phi about the origin [0, 0]
    x_rot = np.cos(phi) * x - np.sin(phi) * y
    y_rot = np.sin(phi) * x + np.cos(phi) * y
    
    return np.array([y_rot, x_rot]) # The first entry is the y coordinate!


def line_fit(traj): # Fits a line to trajectory "traj" which is a collection of points (y, x)
    res = stats.linregress(traj[:, 1], traj[:, 0])
    #                         x           y
    
    return (res.slope, res.intercept, res.stderr, res.rvalue)
    #        slope      intercept      slope std   R test 
    
    def lin(x, a, b):
        return x * a + b
    
    #popt, pcov = so.curve_fit(lin, traj[:, 1], traj[:, 0])
    
    #print(popt[0] - res.slope) # check that both methods give the same results
    #print(np.sqrt(pcov[0][0]) - res.stderr) # check that both methods give the same results
    
    #return (popt[0], popt[1], np.sqrt(pcov[0][0]), res.rvalue)


def manual_regression(traj): # A self-written least-squares method for a line fit
    N = traj.shape[0]
    x_mean = np.average(traj[:, 1])
    y_mean = np.average(traj[:, 0])
    
    bm = np.sum(np.multiply(traj[:, 1] - x_mean, traj[:, 0] - y_mean)) / np.sum(np.power(traj[:, 1] - x_mean, 2))
    am = y_mean - x_mean * bm
    bm_sig = np.sqrt(np.sum(np.power(traj[:, 0] - am - bm * traj[:, 1], 2)) / ((N - 2)*np.sum(np.power(traj[:, 1] - x_mean, 2))))
    # bm is slope, am is intercept here
    
    r2 = 1 - np.sum(np.power(traj[:, 0] - am - bm * traj[:, 1], 2)) / np.sum(np.power(traj[:, 0] - y_mean, 2))
    pearson = np.sum(np.multiply(traj[:, 1] - x_mean, traj[:, 0] - y_mean)) / np.sqrt(np.sum(np.power(traj[:, 0] - y_mean, 2)) * np.sum(np.power(traj[:, 1] - x_mean, 2)))
    
    return (bm, am, bm_sig, r2, pearson)


def default_angle_twostep(traj, com0, default): # rotate the trajectory to angle "default" via two steps
    (ycom0, xcom0) = com0  # "com0" is the footpoint COM, "traj" is the trajectory
    point_i = traj[0] # first point of the trajectory
    point_f = traj[-1] # last point
    # in principle, point_i and point_f should be furthest apart if using the "expand" method.
    gamma = np.arctan((point_f[0] - point_i[0]) / (point_f[1] - point_i[1]))

    angle1 = - gamma + default # angle by which we will rotate
    
    rotated1 = np.zeros((traj.shape[0], 2)) # the rotated trajectoy
    
    for i in range(traj.shape[0]):
        rotated1[i, 0] = rotate(traj[i, 0] - ycom0, traj[i, 1] - xcom0, angle1)[0] + ycom0 # y
        rotated1[i, 1] = rotate(traj[i, 0] - ycom0, traj[i, 1] - xcom0, angle1)[1] + xcom0 # x
    
    slope1 = line_fit(rotated1)[0]
    
    angle2 = - np.arctan(slope1) + default # angle by which we will do the second rotation

    rotated2 = np.zeros((traj.shape[0], 2)) # the twice rotated trajectory
    
    for i in range(traj.shape[0]): # rotate about the footpoint COM again
        rotated2[i, 0] = rotate(rotated1[i, 0] - ycom0, rotated1[i, 1] - xcom0, angle2)[0] + ycom0
        rotated2[i, 1] = rotate(rotated1[i, 0] - ycom0, rotated1[i, 1] - xcom0, angle2)[1] + xcom0

    return rotated2 # the line fitted to "rotated2" should have a coefficient of exactly tan(default)


def default_angle_multistep(traj, com0, default, tolerance): # Rotates the trajectory an arbitrary
    # number of times until its slope is at angle "default", up to some tolerance factor.
    
    (ycom0, xcom0) = com0  # "com0" is the footpoint COM, "traj" is the trajectory
    point_i = traj[0]  # First point of the trajectory
    point_f = traj[-1] # Last point of the trajectory
    # In principle, point_i and point_f should be furthest apart if using the "expand" method.
    if np.abs(point_f[1] - point_i[1]) < 10**(-6.):
        gamma = np.arctan((point_f[0] - point_i[0]) / (point_f[1] - point_i[1] + 10**(-5.)))
    else:
        gamma = np.arctan((point_f[0] - point_i[0]) / (point_f[1] - point_i[1]))
    
    angle1 = - gamma + default # Angle by which we will rotate
    
    rotated1 = np.zeros((traj.shape[0], 2)) # The once-rotated trajectory
    
    for i in range(traj.shape[0]): # Subtract the footpoint COM coordinate so that the "footpoint" of 
                                   # the rotated trajectory is at (0, 0).
        coord_rot = rotate(traj[i, 0] - ycom0, traj[i, 1] - xcom0, angle1)
        rotated1[i, 0] = coord_rot[0] # y
        rotated1[i, 1] = coord_rot[1] # x
    
    slope_temp = line_fit(rotated1)[0]
    
    iterations = 0
    err_token = 0
    
    while(np.abs(slope_temp - np.tan(default)) > tolerance): # Repeat until slope is close enough
        iterations += 1
        angle_temp = - np.arctan(slope_temp) + default
        
        rotated_temp = np.zeros((traj.shape[0], 2)) # The rotated trajectory
    
        for i in range(traj.shape[0]):
            coord_rot = rotate(rotated1[i, 0], rotated1[i, 1], angle_temp)
            rotated_temp[i, 0] = coord_rot[0] # y
            rotated_temp[i, 1] = coord_rot[1] # x
        
        rotated1 = rotated_temp
        slope_temp = line_fit(rotated1)[0]
        
        if iterations > 50:
            print('Final slope is {}'.format(slope_temp))
            err_token = 1
            break
    
    return (rotated1, err_token)
    

####################################################################################################  
# steps_ratio(traj_hor)
# steps_dist_ratio(traj_hor)
# measure_spread(traj, y_fit)
# length1(traj)
# length2(traj)

def steps_ratio(traj_hor): # Gives the ratio of steps forward and backward along the fitted line.
    # A jet is expected to go only in one direction, a flare will go in both.
    # For the formula to make sense, the traj_hor has to be rotated to the horizontal ("default" = 0).
    # This is best achieved by using the "default_angle_multistep" function above.
    
    forward = 0 # the number of steps in the positive direction
    backward = 0 # the number of steps in the opposite direction
    N = traj_hor.shape[0] # length of the trajectory

    for i in range(1, N):
        if traj_hor[i, 1] > traj_hor[i - 1, 1]: # Compare x coordinates
            forward += 1
        else:
            backward += 1
            
    smaller = min(forward, backward)
    larger = max(forward, backward)
    # forward + backward = N - 1
    
    return (smaller / (N - 1), smaller / larger) # Returns two ratios 
    # Range       [0, 1/2]          [0, 1]   
    # Ideal jet      0                0
    # Ideal flare   1/2               1


def steps_dist_ratio(traj_hor): # Ratio of forward / backward travelled distance. Same idea as 
    # "steps_ratio" function except that each step is weighted by the distance over which the COM of 
    # the expanding part moved.
    move_left = 0
    move_right = 0
    N = traj_hor.shape[0] # Length of the trajectory
    
    for i in range(1, N):
        if traj_hor[i, 1] > traj_hor[i-1, 1]:
            move_right += (traj_hor[i, 1] - traj_hor[i-1, 1])
            
        elif traj_hor[i, 1] < traj_hor[i-1, 1]:
            move_left += -(traj_hor[i, 1] - traj_hor[i-1, 1])
    
    shorter = min(move_left, move_right)
    longer = max(move_left, move_right)
    
    return shorter / longer
    # Range       [0, 1]   
    # Ideal jet     0
    # Ideal flare   1
    
def measure_spread(traj, y_fit): # A measure of spread from a horizontal line that is independent
    # of the number of points in the trajectory. If we also divided by the variance in x, we would
    # get the slope standard deviation.
    N = traj.shape[0]
    spread = 0
    
    for i in range(N):
        spread += (traj[i, 0] - y_fit) **2  # y_i**2
        
    return np.sqrt(spread / (N - 2))
    

def length1(traj): # Largest distance between the footpoint and any point on the trajectory
    maxdist = 0

    for i in range(traj.shape[0]):
        if np.sqrt((traj[i, 0] - traj[0, 0])**2 + (traj[i, 1] - traj[0, 1])**2) > maxdist:
            maxdist = np.sqrt((traj[i, 0] - traj[0, 0])**2 + (traj[i, 1] - traj[0, 1])**2)

    return maxdist
    
    
def length2(traj): # Largest distance between any two points on the trajectory.
    maxdist = 0

    for i in range(traj.shape[0]):
        for j in range(i):
            if np.sqrt((traj[i, 0] - traj[j, 0])**2 + (traj[i, 1] - traj[j, 1])**2) > maxdist:
                maxdist = np.sqrt((traj[i, 0] - traj[j, 0])**2 + (traj[i, 1] - traj[j, 1])**2)

    return maxdist
    

####################################################################################################  
# filter_short(objects, min_time)
# filter_small(objects, min_size)
# filter_big(objects, max_size)
# filter_inertia(objects, intensities, max_ratio)
# filter_expand(objects, intensities)

def filter_short(objects, min_time): # Remove all objects that have a lifetime of less than "min_time".
    removed = [] # indices of short-lived objects to be removed
    no = len(objects)
    for i in range(no):
        if len(objects[i]) <= min_time:
            removed.append(i)
    
    for p in sorted(removed, reverse = True): # Remove the short-lived objects.
        del objects[p]  
    
    return (objects, len(removed)) 
    # Returns the remaining objects and the number of removed objects.
    

def filter_small(objects, min_size): # Remove all objects whose time union is smaller than "min_size".
    removed = [] # This filter will be more relevat for high resolution data.
    no = len(objects)
    for i in range(no):
        if len(time_union(objects[i])) < min_size:
            removed.append(i)
    
    for p in sorted(removed, reverse = True):
        del objects[p]
    
    return (objects, len(removed))


def filter_big(objects, max_size): # Remove all objects whose time union is larger than "max_size".
    removed = [] # This filter removes enormous flares which slow down further calculations. For 
    no = len(objects) # example, calculating the time sum/max requires a quadratic loop.
    for i in range(no):
        if len(time_union(objects[i])) > max_size:
            removed.append(i)
    
    for p in sorted(removed, reverse = True):
        del objects[p]
    
    return (objects, len(removed))


def filter_inertia(objects, intensities, max_ratio): # Remove all objects whose ratio of principal
    removed = []  # moments is larger than the maximum allowed ratio, i.e. "max_ratio" ~ 0.2.
    no = len(objects)
    for i in range(no):
        (minor, major) = inertia(objects[i], intensities, 1, 'max') # set power = 1
        if minor / major > max_ratio:
            removed.append(i)
    
    for p in sorted(removed, reverse = True):
        del objects[p]
    
    return (objects, len(removed))


def filter_expand(objects, intensities): # Remove all objects whose "expand" trajectory is shorter 
    # than 3 points. Returns the remaining objects and the number of removed objects.
    removed = []
    no = len(objects)
    for i in range(no):
        traj_i = object_com_masked_traj(objects[i], intensities, 'expand')
        if traj_i.shape[0] < 3:
            removed.append(i)
            
    for p in sorted(removed, reverse = True):
        del objects[p]
    
    return (objects, len(removed))


####################################################################################################
# check_separate(objects, nt)

def check_separate(objects, intensities): # Check if there are any overlaps between objects (there shouldn't be)
    (ny, nx, nt) = intensities.shape
    no = len(objects)
    shapes = [[set() for i in range(no)] for j in range(nt)]
    
    for i in range(no):
        for j in range(len(objects[i][1:])):
            shapes[objects[i][0] + j][i] = objects[i][1 + j]
    
    for j in range(nt):
        for i in range(no):
            for k in range(i):
                if shapes[j][i].intersection(shapes[j][k]) != set():
                    print(shapes[j][i].intersection(shapes[j][k]))
                    print('There are overlaps at time {} between objects {} and {}!'.format(j, i, k))


####################################################################################################  
# make_name(obj_index, letter, batch_no)
# make_limits(objc, margin, intensities)
# make_index(ind)

def make_name(obj_index, letter, batch_no): # Make a name for the file. "Letter" can be 
    # A,B,C,D,E,F,G,H, explanation can be found in the Powerpoint 
    if obj_index < 10:
        return '0000' + str(obj_index) + letter + str(batch_no)
    
    if obj_index < 100:
        return '000' + str(obj_index) + letter + str(batch_no)
    
    if obj_index < 1000:
        return '00' + str(obj_index) + letter + str(batch_no)
    
    if obj_index < 10000:
        return '0' + str(obj_index) + letter + str(batch_no)
    
    else:
        return str(obj_index) + letter + str(batch_no)


def make_limits(objc, margin, intensities): # Make limits for the zoomed-in plots and videos
    # 'margin' is the margin around the object which we set constant for all objects
    (ny, nx, nt) = intensities.shape
    
    objc_union = list(time_union(objc))
    y_list = [objc_union[i][0] for i in range(len(objc_union))] # All y coordinates
    x_list = [objc_union[i][1] for i in range(len(objc_union))] # All x coordinates
    
    ymin = min(y_list)
    ymax = max(y_list)
    xmin = min(x_list)
    xmax = max(x_list)
    
    if ymax + margin < ny:
        y_upper = ymax + margin
    else:  # If an object is close enough to the edge of the frame, there will be fewer pixels than
           # 'margin' around it.
        y_upper = ny
    
    if ymin - margin > 0:
        y_lower = ymin - margin
    else:
        y_lower = 0
        
    if xmax + margin < nx:
        x_upper = xmax + margin
    else: 
        x_upper = nx
    
    if xmin - margin > 0:
        x_lower = xmin - margin
    else:
        x_lower = 0
    
    return (y_lower, y_upper, x_lower, x_upper)


def make_index(ind):
    if ind < 10:
        return '000' + str(ind)
    elif ind < 100:
        return '00' + str(ind)
    elif ind < 1000:
        return '0' + str(ind)

    
####################################################################################################
# sum_histo(prop, minmax, properties, classification, min_length, max_ratio)

def sum_histo(prop, minmax, properties, classification, min_length, max_ratio):
    # How many objects have the value of property "prop" inside the interval (minval, maxval)
    (minval, maxval) = minmax
    sum_jet = 0
    sum_nj = 0
    
    for k in range(properties.shape[0]):
        if properties[k, 4] < max_ratio: # Small enough ratio of principal moments
            if properties[k, 12] >= min_length: # Minimal first expansion time
                if properties[k, 16] == 0 or properties[k, 16] == 2: # Allow for faulty 45-degree line fit
                    if classification[k] == 1:  # The object is a jet
                        if properties[k, prop] < maxval and properties[k, prop] > minval:
                            sum_jet += 1

                    if classification[k] == 0: # The object is a non-jet
                        if properties[k, prop] < maxval and properties[k, prop] > minval:
                            sum_nj += 1
    
    return (sum_jet, sum_nj)


